# Proyecto 2
Ariela Mishaan (22052), Alina Carías (22539), Diego Soto (22737), Ignacio Méndez (22613) y Marcos Díaz

## Librerias

In [3]:
import os
import shutil
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import tensorflow as tf
import json
import mediapipe
import matplotlib
import matplotlib.pyplot as plt
import random

from skimage.transform import resize
from mediapipe.framework.formats import landmark_pb2
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
from matplotlib import animation, rc

ModuleNotFoundError: No module named 'pyarrow'

## Carga de datos

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset_df = pd.read_csv('/kaggle/input/asl-fingerspelling/train.csv')
print("Full train dataset shape is {}".format(dataset_df.shape))

## Exploración

In [ ]:
dataset_df.head()

Cada entrada en el train tiene el nombre del archivo (path), el id del archivo (file_id), el id de la sequencia o la oración (sequence_id), el id del participante (participant_id) y la frase que se representa con el lenguaje de señas (phrase). El file_id indica el archivo que tiene la data de los landmarks para cada frase y el sequence_id es el indice unico de una secuencia dentro de cada archivo de landmarks. 

In [ ]:
archivo = '/Users/arielamishaancohen/Downloads/asl-fingerspelling/supplemental_landmarks/86446671.parquet'

# Leer el archivo parquet
df = pd.read_parquet(archivo)
# Mostrar las primeras filas
df.head()

In [ ]:
print(list(df.columns))

Cada archivo parquet contiene una frase dividida en frames, cada frame es una fila, en las columnas se encuentra la información de las coordenadas. 